# TLC Data Preprocessing (Part 1)
This notebook reads in from landing data and exports to raw, while performing the following preprocessing steps:
1. Missing Value Analysis
2. Invalid Data Analysis

In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.writeLegacyFormat", True)
    .config("spark.driver.memory", "15g")
    .config("spark.sql.parquet.enableVectorizedReader", "false")
    .getOrCreate()
)

24/08/25 01:55:05 WARN Utils: Your hostname, coldbrew.local resolves to a loopback address: 127.0.0.1; using 172.16.119.16 instead (on interface en0)
24/08/25 01:55:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/25 01:55:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sdf_all = spark.read.parquet('../data/landing/tlc_data/')

In [4]:
# Preview the data
num_rows = sdf_all.count()
print("Number of rows", num_rows)
sdf_all.printSchema()

Number of rows 102914393
root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = t

In [5]:
sdf_all.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [6]:
from pyspark.sql import functions as F

# Create a dictionary to hold the counts of missing values (written by ChatGPT)
missing_counts = {col: sdf_all.filter(sdf_all[col].isNull()).count() for col in sdf_all.columns}

# Display the counts
for column, count in missing_counts.items():
    print(f"Column {column} has {count} missing values.")

Column hvfhs_license_num has 0 missing values.
Column dispatching_base_num has 0 missing values.
Column originating_base_num has 28093518 missing values.
Column request_datetime has 0 missing values.
Column on_scene_datetime has 28092882 missing values.
Column pickup_datetime has 0 missing values.
Column dropoff_datetime has 0 missing values.
Column PULocationID has 0 missing values.
Column DOLocationID has 0 missing values.
Column trip_miles has 0 missing values.
Column trip_time has 0 missing values.
Column base_passenger_fare has 0 missing values.
Column tolls has 0 missing values.
Column bcf has 0 missing values.
Column sales_tax has 0 missing values.
Column congestion_surcharge has 0 missing values.
Column airport_fee has 0 missing values.
Column tips has 0 missing values.
Column driver_pay has 0 missing values.
Column shared_request_flag has 0 missing values.
Column shared_match_flag has 0 missing values.
Column access_a_ride_flag has 0 missing values.
Column wav_request_flag has

Only the `originating_base_num` and `on_scene_datetime` columns have missing values. Since these columns are not relevant to our analysis, we can drop them.

In [7]:
sdf_all = sdf_all.drop('originating_base_num', 'on_scene_datetime')

In [8]:
# Identifying invalid data
num_rows_after_drop = sdf_all.filter(
    (F.col('trip_miles') > 0) & (F.col('trip_time') > 0) & (F.col('tips') >= 0)).count()
print("Number of rows dropping", num_rows - num_rows_after_drop)
print("Percentage of rows dropping", (num_rows - num_rows_after_drop) / num_rows)

Number of rows dropping 21604
Percentage of rows dropping 0.0002099220465693268


Only a small percentage of rows are dropped, so we can proceed with dropping these rows.

In [9]:
sdf_all.write.mode('overwrite').parquet('../data/raw/tlc_data')